# Filter Selection

Remove specifics features from dataset.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters and model hyperparameters
Components may declare (and use) these default parameters:
- dataset

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/). <br />
You may also declare custom parameters to set when running an experiment.

Select the hyperparameters and their respective values to be used when training the model:
- features_to_filter

In [ ]:
# parameters
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# hyperparameters
features_to_filter = ["SepalWidthCm", "SepalLengthCm"] #@param {type:"list", label:"Features Para Filtragem", description:"Remove features selecionadas do dataset."}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Wrapping custom transformer

In [ ]:
%%writefile CustomTransformer.py
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class Filter(BaseEstimator, TransformerMixin):
    def __init__(self, features, columns):
        self.features = features
        self.columns = columns
    
    def fit(self, X=None):
        return self
    
    def transform(self, input_series):
        """Remove features from dataset"""
        indexes = np.where(np.in1d(self.columns, self.features))[0]
        return np.delete(input_series, indexes, axis=1)

## Filter dataset

In [ ]:
import pandas as pd
from CustomTransformer import Filter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

# Call changes
pipeline = make_pipeline(
    Filter(features=features_to_filter, columns=columns)
)

# Transform `X`
X = pipeline.transform(X)

# Put back in pd.DataFrame
features_after_pipeline = columns[~np.in1d(columns, features_to_filter), ...]
df = pd.DataFrame(X, columns=features_after_pipeline)
df[target] = pd.Series(y)

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline,
           columns=columns)